In [15]:
import cv2
import mediapipe as mp
import numpy as np
import time
import pandas as pd
from shared_memory_dict import SharedMemoryDict

finger_data_memory = SharedMemoryDict(name='finger-data', size=1024)



#import math
cap = cv2.VideoCapture(0)

mpHands = mp.solutions.hands
hands = mpHands.Hands()

mpDraw = mp.solutions.drawing_utils

In [16]:
import socket
from threading import Timer

In [17]:
pTime = 0
cTime = 0
x_pos = 0
y_pos = 0
sep = 0
loop_counter = 0
scale_taken = False
clenched_taken = False
palm_taken = False
finger_data_memory["Take-Scale"] = False

finger_positions = np.zeros(shape=(22,3))
print(finger_positions[0])

[0. 0. 0.]


In [18]:
class PointCoordinates:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def middle_point(self, other):
        S_O_X_Middle = (self.x + other.x) / 2
        S_O_Y_Middle = (self.y + other.y) / 2
        return(S_O_X_Middle, S_O_Y_Middle)
        
    def point_seperation(self, other):
        S_O_X_Separation = self.x - other.x
        S_O_Y_Separation = self.y - other.y
        S_O_Separation = (((S_O_X_Separation ** 2) + (S_O_Y_Separation ** 2))) ** 0.5
        return(S_O_Separation)

In [19]:
def relative_scale(current_sep, reference_sep):
    try:
        real_sep = current_sep / reference_sep
        return(real_sep)
    except ZeroDivisionError:
        pass

def scale_variable(value, min_in, max_in, min_out, max_out):
    normalized_value = (value - min_in) / (max_in - min_in)
    scaled_value = (normalized_value * (max_out - min_out)) + min_out

    return scaled_value

In [20]:
finger_tops = [4, 8, 12, 16, 20]

finger_reference_points = [3, 7, 11, 15, 19]

finger_bottoms = [1, 5, 9, 13, 17]
#finger_bottoms = [1, 6, 10, 14, 18]
finger_seps = [0, 0, 0, 0, 0]

finger_seps_initial = [0, 0, 0, 0, 0]

finger_seps_raw = [0, 0, 0, 0, 0]

calibration_scales = [0, 0, 0, 0, 0]

calibration_scales_clenched = [0, 0, 0, 0, 0]

current_scales = [0, 0, 0, 0, 0]

current_word = ""

To finish the code, the values from this program need to be written to a CSV file and read simultaneously by another python program as the PC has issues sending through data to the pi as well as doing the motion capture. open file or pandas should do the trick. Also Blue Lobster.

In [21]:
while True: 
    success, img = cap.read()
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)
    if clenched_taken == True:
        finger_data_memory["data"] = finger_seps
    
    for finger in range(0, len(finger_tops)):
        
        finger_1 = finger_positions[finger_tops[finger]][0], finger_positions[finger_tops[finger]][1]
        finger_2 = finger_positions[finger_bottoms[finger]][0], finger_positions[finger_bottoms[finger]][1]
        middle_point = finger_positions[finger_reference_points[finger]][0], finger_positions[finger_reference_points[finger]][1]
        
        point_1 = PointCoordinates(finger_1[0], finger_1[1])
        point_2 = PointCoordinates(finger_2[0], finger_2[1])
        
        cv2.line(img, (int(point_1.x), int(point_1.y)), (int(point_2.x), int(point_2.y)), (0, 255, 0), thickness=2)
        x_pos, y_pos = PointCoordinates.middle_point(point_1, point_2)
        sep = PointCoordinates.point_seperation(point_1, point_2)

        
        point_3 = PointCoordinates(middle_point[0], middle_point[1])
        
        scaling_sep = PointCoordinates.point_seperation(point_1, point_3)
        current_scales[finger] = scaling_sep
        initial_sep = finger_seps_initial[finger]
        finger_seps_raw[finger] = sep
        
        cv2.putText(img, current_word, (200, 50), cv2.FONT_HERSHEY_PLAIN, 2,
        (0, 0, 255), 2)
        if scale_taken == True:
            scaling = relative_scale(scaling_sep, calibration_scales[finger])
            sep = (1 / scaling) * sep
            #sep = (sep / initial_sep) * 180
            sep = scale_variable(sep, calibration_scales_clenched[finger] + 15, finger_seps_initial[finger], 0, 180)
            
            if sep < 20:
                sep = 0
            sep = np.clip(sep, 0, 180)
        
        cv2.putText(img, f"({x_pos}, {y_pos}), Sep = ({sep:.2f})", (int(x_pos), int(y_pos)), cv2.FONT_HERSHEY_PLAIN, 1,
        (255, 0, 255), 1) 
    
        finger_seps[finger] = sep
        
        if sep < 70:
            cv2.circle(img, (int(x_pos), int(y_pos)), 15, (255, 0, 0), cv2.FILLED)
    
    
    
    if scale_taken == True and palm_taken == True and clenched_taken == False:
        if loop_counter - scale_loop_2 == 50:
            clenched_taken = True
            time.sleep(2)
            for n in range(len(calibration_scales_clenched)):
                calibration_scales_clenched[n] = finger_seps_raw[n]
            print(f"CLENCHED: {calibration_scales_clenched}")
            print(finger_seps_initial)
            current_word = ""
         
    if scale_taken == True and palm_taken == False:
        if loop_counter - scale_loop_1 == 50:
            scale_loop_2 = loop_counter
            palm_taken = True
            time.sleep(2)
            for n in range(len(calibration_scales)):
                calibration_scales[n] = current_scales[n]
                finger_seps_initial[n] = finger_seps_raw[n]
            current_word = "Clench your fist."
            
    if scale_taken == False and finger_data_memory["Take-Scale"] == True:
        scale_loop_1 = loop_counter
        scale_taken = True
        current_word = "Spread your hand."
          
    if results.multi_hand_landmarks:
        for handLms in results.multi_hand_landmarks:
            
            #print(temp_data_gathering)
            for id, lm in enumerate(handLms.landmark):
                # Goes through the individual fingers?
                
                # print(id, lm)
                h, w, c = img.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                  #print(id, cx, cy
                  
                for point in range(0,22):
                    if point == id:
                        finger_positions[id][0] = cx
                        finger_positions[id][1] = cy
            mpDraw.draw_landmarks(img, handLms, mpHands.HAND_CONNECTIONS)

    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime

    cv2.putText(img, str(int(fps)), (10, 70), cv2.FONT_HERSHEY_PLAIN, 3,
                (255, 0, 255), 3)
    #cv2.putText(img, f"Ball Coords (x,y,z): ({x_sep}, {int(finger_positions[0][1])}", (10, 450), cv2.FONT_HERSHEY_PLAIN, 1,
    #           (255, 0, 255), 1)
    loop_counter += 1
    #print(finger_seps)
    cv2.imshow("Image", img)
    cv2.waitKey(1)
    

C:\Users\Liam McCord\AppData\Local\Temp\ipykernel_16228\1015987764.py:3: RuntimeWarning: divide by zero encountered in double_scalars
  real_sep = current_sep / reference_sep


CLENCHED: [182.00274723201295, 69.46221994724903, 67.05221845696084, 53.48831648126533, 60.166435825965294]
[182.03571078225283, 151.16877984557524, 162.11107303327555, 155.36408851468863, 130.91982279242512]


C:\Users\Liam McCord\AppData\Local\Temp\ipykernel_16228\3529603055.py:33: RuntimeWarning: divide by zero encountered in double_scalars
  sep = (1 / scaling) * sep


KeyboardInterrupt: 

: 